In [10]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from scipy.cluster.vq import *

Set the dirctory(choose one of the listed 2!!!)
1.default training images&videos
2.self made training images&videos
(For more comments please see the code file, sorry for the inconvence)

In [20]:
trainData_dir = './coursework-data/objects-train/'
testData_dir = './coursework-data/objects-test/'

In [14]:
trainData_dir = './coursework-data/changMadeData/img_train/'
testData_dir = './coursework-data/changMadeData/video_test/'

In [21]:
trainFileNames = glob.glob(trainData_dir+'*.png')
print(len(trainFileNames))
des_list = []
kp_list = []
descriptors_all = []

16


In [22]:
for i in range(len(trainFileNames)):
    img = cv.imread(trainFileNames[i])
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    surf = cv.xfeatures2d.SURF_create(400)
    kp, des = surf.detectAndCompute(gray, None)
    print(len(kp), len(des))
    des_list.append((trainFileNames[i], des))
    kp_list.append((trainFileNames[i], kp))
    for d in des:
        descriptors_all.append(d)

68 68
73 73
128 128
122 122
246 246
430 430
112 112
345 345
890 890
233 233
231 231
352 352
394 394
267 267
340 340
245 245


In [23]:
k = 200
batch_size = np.size(os.listdir(trainData_dir))*3
print(batch_size)
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(descriptors_all)
kmeans.verbose = False
histo_list = []
Y = []

48
Init 1/3 with method: k-means++
Inertia for init 1/3: 3.915662
Init 2/3 with method: k-means++
Inertia for init 2/3: 4.163893
Init 3/3 with method: k-means++


/Users/chang/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=144 should be larger than k=200. Setting it to 3*k
  init_size=init_size)
/Users/chang/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=144 should be larger than k=200. Setting it to 3*k
  init_size=init_size)
/Users/chang/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=144 should be larger than k=200. Setting it to 3*k
  init_size=init_size)


Inertia for init 3/3: 4.522873
Minibatch iteration 1/9400: mean batch inertia: 0.116737, ewa inertia: 0.116737 
Minibatch iteration 2/9400: mean batch inertia: 0.119066, ewa inertia: 0.116787 
Minibatch iteration 3/9400: mean batch inertia: 0.095603, ewa inertia: 0.116333 
Minibatch iteration 4/9400: mean batch inertia: 0.108139, ewa inertia: 0.116157 
Minibatch iteration 5/9400: mean batch inertia: 0.101344, ewa inertia: 0.115839 
Minibatch iteration 6/9400: mean batch inertia: 0.104823, ewa inertia: 0.115603 
Minibatch iteration 7/9400: mean batch inertia: 0.103767, ewa inertia: 0.115349 
Minibatch iteration 8/9400: mean batch inertia: 0.093655, ewa inertia: 0.114884 
Minibatch iteration 9/9400: mean batch inertia: 0.092334, ewa inertia: 0.114401 
[MiniBatchKMeans] Reassigning 24 cluster centers.
Minibatch iteration 10/9400: mean batch inertia: 0.109387, ewa inertia: 0.114293 
Minibatch iteration 11/9400: mean batch inertia: 0.102298, ewa inertia: 0.114036 
Minibatch iteration 12/940

Minibatch iteration 157/9400: mean batch inertia: 0.092949, ewa inertia: 0.086188 
Minibatch iteration 158/9400: mean batch inertia: 0.081107, ewa inertia: 0.086079 
Minibatch iteration 159/9400: mean batch inertia: 0.086325, ewa inertia: 0.086084 
Minibatch iteration 160/9400: mean batch inertia: 0.073849, ewa inertia: 0.085822 
Minibatch iteration 161/9400: mean batch inertia: 0.090462, ewa inertia: 0.085922 
Minibatch iteration 162/9400: mean batch inertia: 0.084424, ewa inertia: 0.085889 
Converged (lack of improvement in inertia) at iteration 162/9400
Computing label assignment and total inertia


In [24]:
for i in range(len(trainFileNames)):
    histo = np.zeros(k)
    nkp = np.size(des_list[i][1])
    print(nkp)

    for d in des_list[i][1]:
        idx = kmeans.predict([d])
        #1/nkp:normalize the histogarm(nkp can be removed due to the less images)
        histo[idx] += 1/nkp
    # plt.bar(list(range(100)), histo)
    # plt.show()
    histo_list.append(histo)

    address = des_list[i][0]
    name = address.rsplit("/", 1)[-1]
    Y.append(name)

4352
4672
8192
7808
15744
27520
7168
22080
56960
14912
14784
22528
25216
17088
21760
15680


In [25]:
X = np.array(histo_list)

In [26]:
videoCap = cv.VideoCapture(testData_dir+"objects-test-2.mov")

#feature matching initialize for labeling the region
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)
frame_count = 0
res = []
surf = cv.xfeatures2d.SURF_create(400)

In [27]:
while (1):
    ret,frame = videoCap.read()
    if ret==True:
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # kp_frame, des_frame = sift.detectAndCompute(frame_gray, None)
        kp_frame, des_frame = surf.detectAndCompute(frame_gray, None)
        print(len(kp_frame))





        if len(kp_frame) >= 10:
            histo_frame = np.zeros(k)
            nkp_frame = np.size(kp_frame)
            for d in des_frame:
                idx_frame = kmeans.predict([d])
                histo_frame[idx_frame] += 1/nkp_frame
            distance_list = []
            #perform NN 
            for i in range(len(trainFileNames)):
                # distance = np.linalg.norm(histo_frame-X[i])

                distance = mean_squared_error(histo_frame, X[i])
                distance_list.append(distance)
            # res = distance_list.index(min(distance_list))
            res = np.argsort(distance_list)[:3]
            print(Y[res[0]], Y[res[1]], Y[res[2]])

            # res = mlp.predict_proba([histo_frame])
            # matchedIndex = np.where(res == np.amax(res))
            # print(matchedIndex[1][0])
            # print(Y[matchedIndex[1][0]])


            template = cv.imread(trainData_dir+Y[res[0]], cv.IMREAD_GRAYSCALE)
            kp_list_template = kp_list[res[0]][1]
            flann_matches = flann.knnMatch(des_list[res[0]][1], des_frame, k=2)
            good_points = []
            for m,n in flann_matches:
                if m.distance < 0.6*n.distance:
                    good_points.append(m)


            template_1 = cv.imread(trainData_dir+Y[res[1]], cv.IMREAD_GRAYSCALE)
            flann_matches_1 = flann.knnMatch(des_list[res[1]][1], des_frame, k=2)
            good_points_1 = []
            for m,n in flann_matches_1:
                if m.distance < 0.6*n.distance:
                    good_points_1.append(m)


            template_2 = cv.imread(trainData_dir+Y[res[2]], cv.IMREAD_GRAYSCALE)
            flann_matches_2 = flann.knnMatch(des_list[res[2]][1], des_frame, k=2)
            good_points_2 = []
            for m,n in flann_matches:
                if m.distance < 0.6*n.distance:
                    good_points_2.append(m)

            good_points_list = [len(good_points), len(good_points_1), len(good_points_2)]
            greatest_index = good_points_list.index(max(good_points_list))

            if greatest_index == 1:
                good_points = good_points_1
                kp_list_template = kp_list[res[1]][1]

            elif greatest_index == 2:
                good_points = good_points_2
                kp_list_template = kp_list[res[2]][1]

            print(good_points_list, greatest_index, len(good_points))









            if len(good_points) >= 10:
                query_pts = np.float32([kp_list_template[m.queryIdx].pt for m in good_points]).reshape(-1,1,2)
                train_pts = np.float32([kp_frame[m.trainIdx].pt for m in good_points]).reshape(-1,1,2)
                matrix, mask = cv.findHomography(query_pts, train_pts, cv.RANSAC, 5.0)
                if matrix is not None:
                    matches_mask = mask.ravel().tolist()

                    h,w = template.shape
                    pts = np.float32([[0,0], [0,h], [w,h], [w,0]]).reshape(-1,1,2)
                    dst = cv.perspectiveTransform(pts, matrix)
                    homography = cv.polylines(frame, [np.int32(dst)], True, (255,0,0), 3)
                    cv.imshow('Video', homography)
                else:
                    cv.imshow('Video', frame)
            else:
                cv.imshow('Video', frame)



            # 	w,h = template.shape[::-1]
            # 	res_template = cv.matchTemplate(frame_gray, template, cv.TM_CCOEFF_NORMED)
            # 	threshold = 0.5
            # 	loc = np.where(res_template>=threshold)
            # 	for pt in zip(*loc[::-1]):
            # 		cv.rectangle(frame, pt, (pt[0]+w, pt[1]+h), (0,0,255), 2)
            # 	cv.imshow('Video', frame)
            # else:

        else:
            cv.imshow('Video', frame)

        if cv.waitKey(1)&0xFF ==ord('q'):
            break

    else:
        break

    frame_count += 1

videoCap.release()
cv.destroyAllWindows()

2455
Branflakes-big.png PG-Tips.png Orgainc.png
[0, 0, 0] 0 0
2415
Branflakes-big.png PG-Tips.png Orgainc.png
[1, 0, 1] 0 1
2355
Branflakes-big.png PG-Tips.png Orgainc.png
[1, 0, 1] 0 1
2306
Branflakes-big.png PG-Tips.png Orgainc.png
[1, 0, 1] 0 1
2359
Branflakes-big.png PG-Tips.png Orgainc.png
[0, 0, 0] 0 0
2350
Branflakes-big.png PG-Tips.png Orgainc.png
[2, 0, 2] 0 2
2358
Branflakes-big.png PG-Tips.png Orgainc.png
[1, 1, 1] 0 1
2329
Branflakes-big.png PG-Tips.png Orgainc.png
[0, 0, 0] 0 0
2400
Branflakes-big.png PG-Tips.png Orgainc.png
[12, 0, 12] 0 12
2343
Branflakes-big.png Orgainc.png PG-Tips.png
[0, 0, 0] 0 0
2397
Branflakes-big.png Orgainc.png PG-Tips.png
[0, 0, 0] 0 0
2475
Branflakes-big.png Orgainc.png PG-Tips.png
[3, 0, 3] 0 3
2507
Branflakes-big.png PG-Tips.png Orgainc.png
[0, 0, 0] 0 0
2580


KeyboardInterrupt: 

# CNN MATCHER(VGG16+imagent+local training images)
By using transfer learning
NOTE: TensorFlow should be installed

In [2]:
import keras
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as pt
import matplotlib.image as mpimg
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, Softmax, Flatten
from keras.utils.np_utils import to_categorical
from keras.optimizers import RMSprop, Adam
import glob
from PIL import Image
import cv2 as cv

Using TensorFlow backend.


In [3]:
trainData_dir = '/Users/chang/Desktop/Master/ImageAndVideo/CW/coursework-data/objects-train/'
testData_dir = '/Users/chang/Desktop/Master/ImageAndVideo/CW/coursework-data/objects-test/'
BATCH_SIZE = 32
MAX_EPOCH = 50

In [4]:
trainFileNames = glob.glob(trainData_dir+'*.png')
print(len(trainFileNames))
num_classes = len(trainFileNames)
labels = []
img_train = []
label_train = []

16


In [5]:
for i in range(len(trainFileNames)):
    img = Image.open(trainFileNames[i]).convert('L')
    img = np.array(img)
    img = cv.cvtColor(cv.resize(img, (224, 224)), cv.COLOR_GRAY2BGR)
    print(img.shape)
    img_train.append(img)
    name = trainFileNames[i].rsplit("/", 1)[-1]
    labels.append(name)
    label_train.append(i)

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


In [6]:
img_train = np.concatenate([arr[np.newaxis] for arr in img_train]).astype('float32')
print(labels)
print(label_train)
label_train = to_categorical(label_train, num_classes=num_classes)
inputs = Input(shape=[224, 224, 3])

['Sainsburys.png', 'Quorn.png', 'Bakewell-Tart.png', 'Quorn-big.png', 'Cheerios-small.png', 'Cheerios-top.png', 'Nestle.png', 'Branflakes-big.png', 'Cheerios.png', 'Branflakes.png', 'Tescos-Kitchen-Foil.png', 'Shredded-Wheat.png', 'Pizza.png', 'Orgainc.png', 'PG-Tips.png', 'Shredded-Wheat-small.png']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [7]:
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_tensor=inputs)
for layer in model_vgg16.layers:
    layer.trainable = False
fla = Flatten()(model_vgg16.output)
fc6 = Dense(4096, activation='relu')(fla)
drop6 = Dropout(rate = 0.5)(fc6)
fc7 = Dense(4096, activation='relu')(drop6)
drop7 = Dropout(rate = 0.5)(fc7)
fc8 = Dense(4096, activation='relu')(drop7)
drop8 = Dropout(rate = 0.5)(fc8)
fc9 = Dense(num_classes, activation='softmax')(drop8)
model = Model(inputs=inputs, outputs=fc9)
optimizer = Adam(lr=0.00001)
model.summary()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(img_train, label_train, batch_size=BATCH_SIZE, epochs=MAX_EPOCH)
model.save(filepath='./retrained_vgg16.h5')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

Epoch 43/50
16/16 [==============================] - 3s 170ms/step - loss: 1.4879 - accuracy: 0.6250
Epoch 44/50
16/16 [==============================] - 3s 171ms/step - loss: 1.9640 - accuracy: 0.6250
Epoch 45/50
16/16 [==============================] - 3s 170ms/step - loss: 3.6930 - accuracy: 0.5000
Epoch 46/50
16/16 [==============================] - 3s 169ms/step - loss: 4.5370 - accuracy: 0.4375
Epoch 47/50
16/16 [==============================] - 3s 173ms/step - loss: 4.5169 - accuracy: 0.4375
Epoch 48/50
16/16 [==============================] - 3s 170ms/step - loss: 2.4435 - accuracy: 0.6875
Epoch 49/50
16/16 [==============================] - 3s 171ms/step - loss: 2.1192 - accuracy: 0.6875
Epoch 50/50
16/16 [==============================] - 3s 174ms/step - loss: 0.9482 - accuracy: 0.6875


In [8]:
model = load_model('retrained_vgg16.h5')
img = load_img(trainData_dir+"Quorn.png", target_size =(224,224))
img_data = img_to_array(img)
print(img_data.shape)
img_data = img_data.reshape((1,)+img_data.shape)
print(img_data.shape)

img_data = preprocess_input(img_data)
prediction = model.predict(img_data)
print(np.argsort(prediction))
print(np.argsort(prediction)[0][:-4:-1])
print(np.argmax(prediction))
prediction = labels[np.argmax(prediction)]
print(prediction)

(224, 224, 3)
(1, 224, 224, 3)
[[ 5 13 14 15 10  9 12  8  0  2  4  7 11  6  3  1]]
[1 3 6]
1
Quorn.png


In [9]:
des_list = []
kp_list = []
descriptors_all = []

In [10]:
for i in range(len(trainFileNames)):
    #load a training image
    img = cv.imread(trainFileNames[i])
    #set the training image to gray scale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    surf = cv.xfeatures2d.SURF_create(400)
    kp, des = surf.detectAndCompute(gray, None)
    print(len(kp), len(des))
    des_list.append((trainFileNames[i], des))
    kp_list.append((trainFileNames[i], kp))
    for d in des:
        descriptors_all.append(d)

68 68
73 73
128 128
122 122
246 246
430 430
112 112
345 345
890 890
233 233
231 231
352 352
394 394
267 267
340 340
245 245


In [13]:
videoCap = cv.VideoCapture(testData_dir+"objects-test-1.mov")

#feature matching initialize for labeling the region
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)
frame_count = 0
res = []

surf = cv.xfeatures2d.SURF_create(400)

In [ ]:
for i in range(100):
    ret,frame = videoCap.read()

In [14]:
while (1):
    ret,frame = videoCap.read()
    if ret==True:
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        kp_frame, des_frame = surf.detectAndCompute(frame_gray, None)
        print(len(kp_frame))

        frame_pre = cv.resize(frame, (224, 224))
        frame_pre = img_to_array(frame_pre)
        frame_pre = frame_pre.reshape((1,)+frame_pre.shape)
        frame_pre = preprocess_input(frame_pre)
        prediction = model.predict(frame_pre)
        res = np.argsort(prediction)[0][:-4:-1]
        print(len(res), res)
        prediction = labels[np.argmax(prediction)]
        print(prediction)
        print(labels[res[0]], labels[res[1]], labels[res[2]])




        if len(kp_frame) >= 10:
            template = cv.imread(trainData_dir+labels[res[0]], cv.IMREAD_GRAYSCALE)
            kp_list_template = kp_list[res[0]][1]
            flann_matches = flann.knnMatch(des_list[res[0]][1], des_frame, k=2)
            good_points = []
            for m,n in flann_matches:
                if m.distance < 0.6*n.distance:
                    good_points.append(m)


#             template_1 = cv.imread(trainData_dir+labels[res[1]], cv.IMREAD_GRAYSCALE)
#             flann_matches_1 = flann.knnMatch(des_list[res[1]][1], des_frame, k=2)
#             good_points_1 = []
#             for m,n in flann_matches_1:
#                 if m.distance < 0.6*n.distance:
#                     good_points_1.append(m)


#             template_2 = cv.imread(trainData_dir+labels[res[2]], cv.IMREAD_GRAYSCALE)
#             flann_matches_2 = flann.knnMatch(des_list[res[2]][1], des_frame, k=2)
#             good_points_2 = []
#             for m,n in flann_matches:
#                 if m.distance < 0.6*n.distance:
#                     good_points_2.append(m)

#             good_points_list = [len(good_points), len(good_points_1), len(good_points_2)]
#             greatest_index = good_points_list.index(max(good_points_list))

#             if greatest_index == 1:
#                 good_points = good_points_1
#                 kp_list_template = kp_list[res[1]][1]

#             elif greatest_index == 2:
#                 good_points = good_points_2
#                 kp_list_template = kp_list[res[2]][1]

#             print(good_points_list, greatest_index, len(good_points))


            if len(good_points) >= 10:
                query_pts = np.float32([kp_list_template[m.queryIdx].pt for m in good_points]).reshape(-1,1,2)
                train_pts = np.float32([kp_frame[m.trainIdx].pt for m in good_points]).reshape(-1,1,2)
                matrix, mask = cv.findHomography(query_pts, train_pts, cv.RANSAC, 5.0)
                if matrix is not None:
                    matches_mask = mask.ravel().tolist()

                    h,w = template.shape
                    pts = np.float32([[0,0], [0,h], [w,h], [w,0]]).reshape(-1,1,2)
                    dst = cv.perspectiveTransform(pts, matrix)
                    homography = cv.polylines(frame, [np.int32(dst)], True, (255,0,0), 3)
                    cv.imshow('Video', homography)
                else:
                    cv.imshow('Video', frame)
            else:
                cv.imshow('Video', frame)

        else:
            cv.imshow('Video', frame)

        if cv.waitKey(1)&0xFF ==ord('q'):
            break

    else:
        break

    frame_count += 1


videoCap.release()
cv.destroyAllWindows()

0
3 [ 1 11  4]
Quorn.png
Quorn.png Shredded-Wheat.png Cheerios-small.png
0
3 [ 4  1 13]
Cheerios-small.png
Cheerios-small.png Quorn.png Orgainc.png
153
3 [ 1  7 13]
Quorn.png
Quorn.png Branflakes-big.png Orgainc.png
162
3 [ 7  1 13]
Branflakes-big.png
Branflakes-big.png Quorn.png Orgainc.png
161
3 [7 1 3]
Branflakes-big.png
Branflakes-big.png Quorn.png Quorn-big.png
159
3 [ 7  1 13]
Branflakes-big.png
Branflakes-big.png Quorn.png Orgainc.png
156
3 [ 1  7 13]
Quorn.png
Quorn.png Branflakes-big.png Orgainc.png
163
3 [ 1  7 13]
Quorn.png
Quorn.png Branflakes-big.png Orgainc.png
163
3 [7 1 3]
Branflakes-big.png
Branflakes-big.png Quorn.png Quorn-big.png
164
3 [ 7  1 11]
Branflakes-big.png
Branflakes-big.png Quorn.png Shredded-Wheat.png
162
3 [ 7  1 11]
Branflakes-big.png
Branflakes-big.png Quorn.png Shredded-Wheat.png
170
3 [ 7  1 11]
Branflakes-big.png
Branflakes-big.png Quorn.png Shredded-Wheat.png
158
3 [ 7  1 11]
Branflakes-big.png
Branflakes-big.png Quorn.png Shredded-Wheat.png
159
3 

0
3 [11  1  8]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png Cheerios.png
0
3 [11  1  7]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png Branflakes-big.png
0
3 [11  8 15]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png Shredded-Wheat-small.png
0
3 [11  1 13]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png Orgainc.png
0
3 [11  1 13]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png Orgainc.png
0
3 [11  7  1]
Shredded-Wheat.png
Shredded-Wheat.png Branflakes-big.png Quorn.png
0
3 [11 13  1]
Shredded-Wheat.png
Shredded-Wheat.png Orgainc.png Quorn.png
0
3 [11 14  1]
Shredded-Wheat.png
Shredded-Wheat.png PG-Tips.png Quorn.png
0
3 [11  8 14]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png PG-Tips.png
0
3 [11  8  1]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png Quorn.png
0
3 [11  8  1]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png Quorn.png
0
3 [11  8  1]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png Quorn.png
0
3 [11  1  8]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png 

3 [11  1 14]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png PG-Tips.png
0
3 [11  1 14]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png PG-Tips.png
0
3 [11  1 14]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png PG-Tips.png
0
3 [11  1 13]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png Orgainc.png
0
3 [11  1 13]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png Orgainc.png
0
3 [11  1 14]
Shredded-Wheat.png
Shredded-Wheat.png Quorn.png PG-Tips.png
0
3 [ 1 11 13]
Quorn.png
Quorn.png Shredded-Wheat.png Orgainc.png
0
3 [ 1 11  4]
Quorn.png
Quorn.png Shredded-Wheat.png Cheerios-small.png
6
3 [13  1 11]
Orgainc.png
Orgainc.png Quorn.png Shredded-Wheat.png
19
3 [1 3 5]
Quorn.png
Quorn.png Quorn-big.png Cheerios-top.png
30
3 [1 5 4]
Quorn.png
Quorn.png Cheerios-top.png Cheerios-small.png
54
3 [ 1 15  5]
Quorn.png
Quorn.png Shredded-Wheat-small.png Cheerios-top.png
69
3 [ 1 13  4]
Quorn.png
Quorn.png Orgainc.png Cheerios-small.png
88
3 [1 3 4]
Quorn.png
Quorn.png Quorn-big.png Cheerios-small.png
1

3 [ 8 11 15]
Cheerios.png
Cheerios.png Shredded-Wheat.png Shredded-Wheat-small.png
304
3 [ 8 11 15]
Cheerios.png
Cheerios.png Shredded-Wheat.png Shredded-Wheat-small.png
332
3 [11  8 15]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png Shredded-Wheat-small.png
345
3 [ 8 11  3]
Cheerios.png
Cheerios.png Shredded-Wheat.png Quorn-big.png
328
3 [11  8  3]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png Quorn-big.png
326
3 [6 3 8]
Nestle.png
Nestle.png Quorn-big.png Cheerios.png
310
3 [ 1 11  3]
Quorn.png
Quorn.png Shredded-Wheat.png Quorn-big.png
315
3 [11  8  1]
Shredded-Wheat.png
Shredded-Wheat.png Cheerios.png Quorn.png
297
3 [5 3 4]
Cheerios-top.png
Cheerios-top.png Quorn-big.png Cheerios-small.png
286
3 [ 8  6 11]
Cheerios.png
Cheerios.png Nestle.png Shredded-Wheat.png
312
3 [ 3 11  6]
Quorn-big.png
Quorn-big.png Shredded-Wheat.png Nestle.png
342
3 [ 3  8 11]
Quorn-big.png
Quorn-big.png Cheerios.png Shredded-Wheat.png
340
3 [ 3 11  8]
Quorn-big.png
Quorn-big.png Shredded-Wheat.pn

KeyboardInterrupt: 